# TDA de Cambios en Precios y Oferta-Demanda de Productos Perecederos
Author: A. Ramirez-Morales (andres.ramirez@tec.mx)


En este notebook se implementa:
1. Generación avanzada de datos con Monte Carlo
2. Análisis TDA completo (homología persistente + Mapper)
3. Detección de cambios estructurales con ventanas deslizantes y Takens embedding
4. Modelos predictivos híbridos (TDA + ML)

### 1. Importación de librerías

In [1]:
!pip install giotto-tda
!pip install imblearn

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from gtda.homology import VietorisRipsPersistence
from gtda.plotting import plot_diagram
from gtda.mapper import (
    make_mapper_pipeline,
    Projection,
    plot_static_mapper_graph
)
from sklearn.cluster import DBSCAN
from imblearn.over_sampling import SMOTE
import networkx as nx
np.random.seed(42)
plt.style.use('ggplot')

### 2. Generación de datos

Actividades/preguntas:
1. Definir los siguientes conceptos: producto perecedero, evento de mercado, brokers, estrategia de broker, tipos de estrageias de broker, variables climaticas, estacionalidad, calidad de producto, elasticidad cruzada
2. Comentar (describir) las líneas más importantes y escribir documentación adecuada para la clase `DatosAgricolas`
3. En la clase `DatosAgricolas` hay varios parámetros que usted puede ajustar, identifíquelos e interprete ssu efecto en los datos.
4. Describa el outcome que espera de la clase `DatosAgricolas`
5. Discuta cualesquiera otras informaciones en `DataAgricolas` que puedan ser relevantes.

In [ ]:
class DatosAgricolas:
    """
    Simulación de datos para productos agrícolas perecederos
    con eventos de mercado, comportamientos de brokers y variables climáticas.
    """
    def __init__(self, n_dias=1000):
        self.n_dias = n_dias
        self.fechas = pd.date_range(start="2023-01-01", periods=n_dias, freq='D')
        self.df = None
        self.eventos = None

    def generar_base(self):
        """
        Genera datos base para los productos
        """
        # precios base con tendencias estacionales
        tiempo = np.linspace(0, 10, self.n_dias)
        estacionalidad = np.sin(2 * np.pi * tiempo / 365) * 0.2 # Se espera un comportamiento repetitivo 

        # frambuesa - producto altamente perecedero
        precio_frambuesa = np.exp(np.random.normal(2 + estacionalidad, 0.3))
        volumen_frambuesa = np.random.normal(500, 100) + np.random.normal(0, 50) * estacionalidad

        # aguacate - menos perecedero
        precio_aguacate = np.exp(np.random.normal(2.5 + estacionalidad*0.5, 0.2))
        volumen_aguacate = np.random.normal(800, 150) - np.random.normal(0, 70) * estacionalidad

        # chile seco - producto estable
        precio_chile = np.exp(np.random.normal(2.2, 0.15))
        volumen_chile = np.random.normal(300, 50)

        # variables climáticas
        # Se toma una medición promedio a la cual se le agregan variaciones normales a lo largo del tiempo 
        temperatura = 25 + 10 * np.sin(2 * np.pi * tiempo / 365) + np.random.normal(0, 3)
        humedad = 60 + 20 * np.sin(2 * np.pi * tiempo / 365 + np.pi/2) + np.random.normal(0, 5)

        # calidad (depende de temperatura y humedad)
        # Establece un límite para los valores en la calidad
        calidad_frambuesa = np.clip(0.9 - 0.005*(temperatura-25)**2 - 0.002*(humedad-70)**2 + np.random.normal(0, 0.05), 0.5, 1.0)
        calidad_aguacate = np.clip(0.95 - 0.003*(temperatura-22)**2 + np.random.normal(0, 0.03), 0.7, 1.0)
        calidad_chile = np.clip(0.85 - 0.001*(humedad-50)**2 + np.random.normal(0, 0.04), 0.6, 1.0)

        # datos sin modificaciones de brokers
        datos = {
            'fecha': self.fechas,
            'precio_frambuesa': precio_frambuesa,
            'volumen_frambuesa': volumen_frambuesa,
            'calidad_frambuesa': calidad_frambuesa,
            'precio_aguacate': precio_aguacate,
            'volumen_aguacate': volumen_aguacate,
            'calidad_aguacate': calidad_aguacate,
            'precio_chile': precio_chile,
            'volumen_chile': volumen_chile,
            'calidad_chile': calidad_chile,
            'temperatura': temperatura,
            'humedad': humedad,
        }

        return pd.DataFrame(datos)

    def agregar_eventos_mercado(self, df):
        """
        Agrega eventos de mercado simulados
        """
        eventos = np.zeros(self.n_dias)

        # frambuesa (perecedero)
        for i in range(100, self.n_dias, 180):
            duracion = np.random.randint(5, 15)
            intensidad = np.random.uniform(0.4, 0.7)

            df.loc[i:i+duracion, 'precio_frambuesa'] *= intensidad
            df.loc[i:i+duracion, 'volumen_frambuesa'] *= np.random.uniform(1.2, 1.8)
            eventos[i:i+duracion] = 1  # Evento tipo 1: sobreoferta

        # aguacate (especulación)
        for i in range(150, self.n_dias, 220):
            duracion = np.random.randint(10, 20)
            intensidad = np.random.uniform(1.3, 1.8)

            df.loc[i:i+duracion, 'precio_aguacate'] *= intensidad
            df.loc[i:i+duracion, 'volumen_aguacate'] *= np.random.uniform(0.6, 0.9)
            eventos[i:i+duracion] = 2  # Evento tipo 2: especulación

        # climáticos extremos
        for i in range(200, self.n_dias, 300):
            duracion = np.random.randint(7, 14)

            df.loc[i:i+duracion, 'temperatura'] += np.random.uniform(5, 10)
            df.loc[i:i+duracion, 'humedad'] -= np.random.uniform(15, 25)
            eventos[i:i+duracion] = 3  # Evento tipo 3: clima extremo

        return df, eventos

    def simular_comportamiento_brokers(self, df):
        """
        Simula el comportamiento estratégico de brokers
        """
        # estrategias de brokers basadas en condiciones de mercado
        df['estrategia_broker'] = 'neutral'

        # identificar condiciones para diferentes estrategias
        mask_sobreoferta = (df['precio_frambuesa'].pct_change(5) < -0.15)
        mask_especulacion = (df['precio_aguacate'].pct_change(5) > 0.2)
        mask_clima_extremo = (df['temperatura'] > 32) | (df['humedad'] < 40)

        df.loc[mask_sobreoferta, 'estrategia_broker'] = 'liquidacion'
        df.loc[mask_especulacion, 'estrategia_broker'] = 'acaparamiento'
        df.loc[mask_clima_extremo, 'estrategia_broker'] = 'precaucion'

        return df

    def generar_datos(self):
        """
        Genera el conjunto completo de datos simulados
        """
        df = self.generar_base()
        df, eventos = self.agregar_eventos_mercado(df)
        df = self.simular_comportamiento_brokers(df)

        # agregar elasticidad cruzada simulada
        df['precio_fresa'] = df['precio_frambuesa'] * np.random.uniform(0.8, 1.2)
        df.loc[df['precio_frambuesa'] > df['precio_frambuesa'].quantile(0.8), 'precio_fresa'] *= 1.1

        self.df = df
        self.eventos = eventos
        return df, eventos

### 2.1 Genear y visualizar datos

Actividades/preguntas:
1. Genere los datos y guardélos en una base de datos con formato csv.
2. Grafique los datos más relevantes a su consideración.
3. Juegue con los parámetros que considere mas relvantes dentro de la clase `DatosAgricolas`
4. Dé una interpretación
